<a href="https://colab.research.google.com/github/Akos1993/Akos1993/blob/main/Movie_Recommendation_Algorythm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generating Dummy Data to work with

In [ ]:
import random
!pip install faker
from faker import Faker
import csv

# Initialize Faker
fake = Faker()

# Generate Users data
def generate_users(num_users=100):
    users = []
    for user_id in range(1, num_users + 1):
        name = fake.first_name()
        age = random.randint(18, 65)
        gender = random.choice(['Male', 'Female'])
        users.append([user_id, name, age, gender])
    return users

# Generate Movies data
def generate_movies(num_movies=100):
    genres = ['Sci-Fi', 'Action', 'Drama', 'Romance', 'Thriller', 'Comedy', 'Animation', 'Crime', 'Musical']
    movies = []
    for movie_id in range(1, num_movies + 1):
        title = fake.sentence(nb_words=3).strip('.')
        genre = random.choice(genres)
        release_year = random.randint(1980, 2025)
        movies.append([movie_id, title, genre, release_year])
    return movies

# Generate Ratings data
def generate_ratings(num_ratings=100, num_users=100, num_movies=100):
    ratings = []
    for rating_id in range(1, num_ratings + 1):
        user_id = random.randint(1, num_users)
        movie_id = random.randint(1, num_movies)
        rating = random.randint(1, 5)
        timestamp = fake.date_time_this_decade()
        ratings.append([rating_id, user_id, movie_id, rating, timestamp])
    return ratings

# Write data to CSV files
def write_to_csv(filename, data, headers):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)

# Generate and save Users data
users = generate_users()
write_to_csv('users.csv', users, ['UserID', 'Name', 'Age', 'Gender'])

# Generate and save Movies data
movies = generate_movies()
write_to_csv('movies.csv', movies, ['MovieID', 'Title', 'Genre', 'ReleaseYear'])

# Generate and save Ratings data
ratings = generate_ratings()
write_to_csv('ratings.csv', ratings, ['RatingID', 'UserID', 'MovieID', 'Rating', 'Timestamp'])

print("Dummy data generated and saved to CSV files: users.csv, movies.csv, ratings.csv")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.0 MB/s eta 0:00:00
Dummy data generated and saved to CSV files: users.csv, movies.csv, ratings.csv


In [ ]:
Install Pandas library for manipulating data

In [ ]:
!pip install pandas

Clear data

In [ ]:
import pandas as pd

def clean_data(file_path, drop_duplicates=True, fill_missing=None):
    """
    Cleans a dataset by removing duplicates and dealing with incomplete data.

    Parameters:
        file_path (str): Path to the CSV file to be cleaned.
        drop_duplicates (bool): Whether to drop duplicate rows. Default is True.
        fill_missing (dict): A dictionary specifying column names and values to fill missing data.
                             Example: {'Age': 25, 'Rating': 3}
                             If None, drops rows with missing data. Default is None.

    Returns:
        pd.DataFrame: A cleaned pandas DataFrame.
    """
    # Load the dataset
    df = pd.read_csv(file_path)

    # Remove duplicates
    if drop_duplicates:
        df = df.drop_duplicates()

    # Handle missing data
    if fill_missing:
        df = df.fillna(fill_missing)  # Fill missing values with provided dictionary
    else:
        df = df.dropna()  # Drop rows with missing values

    return df

# Example usage for users.csv
cleaned_users = clean_data(
    file_path='users.csv',
    drop_duplicates=True,
    fill_missing={'Age': 25, 'Gender': 'Unknown'}  # Fill missing 'Age' with 25 and 'Gender' with 'Unknown'
)

# Example usage for movies.csv
cleaned_movies = clean_data(
    file_path='movies.csv',
    drop_duplicates=True,
    fill_missing={'Genre': 'Unknown', 'ReleaseYear': 2000}  # Fill missing 'Genre' and 'ReleaseYear'
)

# Example usage for ratings.csv
cleaned_ratings = clean_data(
    file_path='ratings.csv',
    drop_duplicates=True,
    fill_missing={'Rating': 3}  # Fill missing 'Rating' with 3
)

# Display cleaned data samples
print("Cleaned Users Data:")
print(cleaned_users.head())

print("\nCleaned Movies Data:")
print(cleaned_movies.head())

print("\nCleaned Ratings Data:")
print(cleaned_ratings.head())


Cleaned Users Data:
   UserID     Name  Age  Gender
0       1  Michael   22    Male
1       2   Ashley   65  Female
2       3    Holly   26  Female
3       4    Traci   29  Female
4       5   Wesley   64  Female

Cleaned Movies Data:
   MovieID                     Title     Genre  ReleaseYear
0        1               Region goal    Sci-Fi         1992
1        2           Certainly group  Thriller         1996
2        3                Goal issue   Romance         2014
3        4              Easy explain     Crime         2013
4        5  Nearly fire sound method    Comedy         2006

Cleaned Ratings Data:
   RatingID  UserID  MovieID  Rating                   Timestamp
0         1      79       81       1  2021-04-19 03:18:31.839656
1         2       5       10       2  2022-07-30 06:16:41.788621
2         3      35       49       3  2022-06-20 21:35:50.899465
3         4      94       50       3  2021-06-28 06:25:36.359843
4         5      85       87       1  2023-12-15 19:54:22.

In [ ]:
def find_highest_rated_movies(movies_df, ratings_df):
    """
    Finds the highest-rated movies based on average ratings.

    Parameters:
        movies_df (pd.DataFrame): DataFrame containing movie information with columns ['MovieID', 'Title'].
        ratings_df (pd.DataFrame): DataFrame containing ratings with columns ['MovieID', 'Rating'].

    Returns:
        pd.DataFrame: DataFrame containing the highest-rated movies with their average ratings.
    """
    # Calculate the average rating for each movie
    avg_ratings = ratings_df.groupby('MovieID')['Rating'].mean().reset_index()
    avg_ratings.rename(columns={'Rating': 'AverageRating'}, inplace=True)

    # Merge with the movies DataFrame to get movie titles
    highest_rated = pd.merge(avg_ratings, movies_df, on='MovieID')

    # Sort by AverageRating in descending order
    highest_rated = highest_rated.sort_values(by='AverageRating', ascending=False)

    return highest_rated

# Process data to find the highest-rated movies
highest_rated_movies = find_highest_rated_movies(movies_df, ratings_df)

# Display the results
print("Top Rated Movies:")
print(highest_rated_movies.head(10))  # Display the top 10 highest-rated movies


Top Rated Movies:
    MovieID  AverageRating                     Title     Genre  ReleaseYear
58      100            5.0              Leader maybe  Thriller         2017
43       78            5.0     Than statement family    Sci-Fi         1982
21       39            5.0      Across serve include  Thriller         2019
1         4            5.0              Easy explain     Crime         2013
24       46            5.0                Quite real     Drama         2011
42       77            5.0            Investment act    Comedy         1981
35       63            5.0            Although learn   Musical         2000
5        12            5.0                  Resource     Crime         2010
7        16            4.0            Position blood   Romance         2005
10       22            4.0  Individual respond piece     Drama         2006


top rated per genre

In [ ]:
def get_highest_rated_movies_by_genre(genre, movies_df, ratings_df, top_n=10):
    """
    Returns the highest-rated movies for a specific genre.

    Parameters:
        genre (str): The genre to filter movies by (e.g., 'Sci-Fi').
        movies_df (pd.DataFrame): DataFrame containing movie information with columns ['MovieID', 'Title', 'Genre'].
        ratings_df (pd.DataFrame): DataFrame containing ratings with columns ['MovieID', 'Rating'].
        top_n (int): The number of top movies to return. Default is 10.

    Returns:
        pd.DataFrame: DataFrame containing the highest-rated movies for the specified genre.
    """
    # Filter movies by the specified genre
    genre_movies = movies_df[movies_df['Genre'].str.contains(genre, case=False, na=False)]

    # Merge ratings with the genre-specific movies
    genre_ratings = pd.merge(genre_movies, ratings_df, on='MovieID')

    # Sort movies by Rating in descending order
    highest_rated = genre_ratings.sort_values(by='Rating', ascending=False).drop_duplicates('MovieID').head(top_n)

    # Return the top-rated movies
    return highest_rated[['MovieID', 'Title', 'Genre', 'Rating']]

# Example usage
# Load the dummy data
movies_df = pd.read_csv('movies.csv')  # Movies data from earlier
ratings_df = pd.read_csv('ratings.csv')  # Ratings data from earlier

# Get the top 10 Sci-Fi movies
top_sci_fi_movies = get_highest_rated_movies_by_genre('Sci-Fi', movies_df, ratings_df, top_n=10)
print("Top Sci-Fi Movies:")
print(top_sci_fi_movies)



Top Sci-Fi Movies:
   MovieID                        Title   Genre  Rating
4       78        Than statement family  Sci-Fi       5
6       89   Avoid increase cell former  Sci-Fi       4
1       53  Organization four interview  Sci-Fi       3
0       51              Her time assume  Sci-Fi       2
3       66      Star knowledge cultural  Sci-Fi       1
5       87              Fight now staff  Sci-Fi       1


Personalised recomendations based on User data

In [ ]:
def get_user_recommendations_top_genres(user_id, movies_df, ratings_df, top_n=10):
    """
    Generates movie recommendations based on the user's top 3 favorite genres.

    Parameters:
        user_id (int): The ID of the user to generate recommendations for.
        movies_df (pd.DataFrame): DataFrame containing movie information with columns ['MovieID', 'Title', 'Genre'].
        ratings_df (pd.DataFrame): DataFrame containing ratings with columns ['UserID', 'MovieID', 'Rating'].
        top_n (int): The number of top movie recommendations per genre. Default is 10.

    Returns:
        dict: A dictionary with genres as keys and DataFrames of recommended movies as values.
    """
    # Get the user's ratings
    user_ratings = ratings_df[ratings_df['UserID'] == user_id]

    # Merge the user's ratings with movies to identify genres of rated movies
    user_movies = pd.merge(user_ratings, movies_df, on='MovieID')

    # Identify the user's top 3 favorite genres based on highest average ratings
    top_genres = (
        user_movies.groupby('Genre')['Rating']
        .mean()
        .sort_values(ascending=False)
        .head(3)  # Get the top 3 genres
        .index.tolist()
    )

    # Generate recommendations for each top genre
    recommendations = {}
    for genre in top_genres:
        recommendations[genre] = get_highest_rated_movies_by_genre(
            genre=genre,
            movies_df=movies_df,
            ratings_df=ratings_df,
            top_n=top_n
        )

    return recommendations

# Example usage
# Dummy data (replace with your actual data)
movies_df = pd.read_csv('movies.csv')  # Movies data
ratings_df = pd.read_csv('ratings.csv')  # Ratings data

# Generate recommendations for User ID 1
user_recommendations = get_user_recommendations_top_genres(1, movies_df, ratings_df, top_n=10)

# Display recommendations for each genre
for genre, recommendations in user_recommendations.items():
    print(f"Top movies for genre '{genre}':")
    print(recommendations)
    print()


Top movies for genre 'Musical':
    MovieID                     Title    Genre  Rating
4        27       Activity make speak  Musical       5
12       63            Although learn  Musical       5
8        29                     Check  Musical       4
9        38                Cause rule  Musical       4
11       49                  Mrs case  Musical       4
16       81               Fly mention  Musical       4
14       79           Really teach by  Musical       3
1        10  Available involve others  Musical       3
13       76                      Ball  Musical       1
18       88      Individual much fund  Musical       1

